In [151]:
%matplotlib inline
import logging
import Euclid
import matplotlib.pyplot as plt
import astropy.units as u
import numpy as np

from matplotlib import colors, patches, cm
from astropy.table import Table, Column
from tqdm.notebook import tqdm
from scipy.stats import gaussian_kde
from scipy.signal import find_peaks
from scikitplot.metrics import plot_confusion_matrix
from IPython.display import set_matplotlib_formats

%elogin
%erun SIM_Tools

from SimSpectra.SimSpectra import SimGalaxySpectra
from ST_DM_MDBTools.Mdb import Mdb

plt.rcParams['figure.figsize'] = (10, 6)
set_matplotlib_formats('pdf')

In [2]:
t = Table.read('/home/aalvarez/Work/Data/SC8/run5/pp_rerun_20210129_with_tu_SNR10.fits')

In [3]:
spec = SimGalaxySpectra(
    sedlib_filename='/home/aalvarez/Work/Data/SC8/TU/Lib/EUC-TEST-GSEDLIB-2015-03-18T171100.000.fits',
    filter_filename='/home/aalvarez/Work/Data/SC8/TU/Lib/EUC-TEST-FILSDSSR01-2015-02-02T203100.000.fits',
    extlib_filename='/home/aalvarez/Work/Data/SC8/TU/Lib/EUC-TEST-GEXTLAWLIB-2015-03-18T203100.000.fits',
    #extmap_filename=[
    #    '/home/aalvarez/Work/Data/SC8/TU/Lib/SFD_dust_1024_ngp.fits',
    #    '/home/aalvarez/Work/Data/SC8/TU/Lib/SFD_dust_1024_sgp.fits',
    #],
    extmap_filename=['/home/aalvarez/Work/Data/SC8/TU/Lib/HFI_CompMap_ThermalDustModel_2048_R1.20_V2.fits'],
    norm_sed='AB.sed',
    wmin=   500E-10,
    wmax=100000E-10,
    nwl=2000
)

INFO    : Use internal AB reference spectrum
2021-02-09T10:49:09CET SimSpectra  INFO : Use internal AB reference spectrum


In [4]:
def get_spectra(obj, sed_template):
    spectra = spec.get_spectra(
        ra=float(obj['RA_MAG']), dec=float(obj['DEC_MAG']), refmag=float(obj['REF_MAG']), absmag=float(obj['REF_MAG_ABS']),
        z=float(obj['Z_OBS']), ised=sed_template, ebv=float(obj['EBV']), extlaw=float(obj['EXT_LAW']), vacuum=True,
        lines={
            'halpha': float(obj['HALPHA_LOGFLAM_EXT_MAG']),
            'hbeta': float(obj['HBETA_LOGFLAM_EXT_MAG']),
            'O2': float(obj['O2_LOGFLAM_EXT_MAG']),
            'O3': float(obj['O3_LOGFLAM_EXT_MAG']),
            'N2': float(obj['N2_LOGFLAM_EXT_MAG']),
            'S2': float(obj['S2_LOGFLAM_EXT_MAG'])
        },
    )
    # Spectra is in Watts
    spectra *= 1e-7
    return np.stack([spec.wgrid.to(u.Angstrom).value, spectra], axis=-1)

In [176]:
st = np.arange(0, 30.)

fig, axes = plt.subplots(ncols=3, nrows=10, figsize=(20, 20))
for i, ax in enumerate(axes.ravel()):
    spectra = get_spectra(t[0], st[i])
    ax.plot(spectra[:,0], spectra[:,1]/spectra[:,1].sum())
    ax.set_title(st[i])
plt.tight_layout()

INFO    : Compute spectra without lensing magnification
2021-02-09T14:08:19CET SimSpectra  INFO : Compute spectra without lensing magnification
INFO    : Compute spectra without lensing magnification
2021-02-09T14:08:19CET SimSpectra  INFO : Compute spectra without lensing magnification
INFO    : Compute spectra without lensing magnification
2021-02-09T14:08:19CET SimSpectra  INFO : Compute spectra without lensing magnification
INFO    : Compute spectra without lensing magnification
2021-02-09T14:08:19CET SimSpectra  INFO : Compute spectra without lensing magnification
INFO    : Compute spectra without lensing magnification
2021-02-09T14:08:19CET SimSpectra  INFO : Compute spectra without lensing magnification
INFO    : Compute spectra without lensing magnification
2021-02-09T14:08:19CET SimSpectra  INFO : Compute spectra without lensing magnification
INFO    : Compute spectra without lensing magnification
2021-02-09T14:08:19CET SimSpectra  INFO : Compute spectra without lensing magnif

<Figure size 1440x1440 with 30 Axes>

Roughly

* From 0 to 8 elliptical
* From 9 to 19 spiral
* From 20 to 30 SB

In [43]:
SPIRAL_LMIN = 9
SB_LMIN = 19.5

t['TRUE_GALCLASS'] = 0
t['TRUE_GALCLASS'][(t['SED_TEMPLATE'] >= SPIRAL_LMIN)] = 1
t['TRUE_GALCLASS'][(t['SED_TEMPLATE'] >= SB_LMIN)] = 2
mask = t['FLAG']==0

In [178]:
plt.figure()
density_plot(t[mask]['REF_MAG_ABS'], t[mask]['PHZ_PP_MODE_LOG_STELLARMASS'])
plt.xlabel('Reference Magnitude')
plt.ylabel('Log Stellarmass')
plt.show()

<Figure size 720x432 with 1 Axes>

In [179]:
def density_plot(a, b, ax=None):
    if not ax:
        ax = plt.gca()
    xy = np.vstack([a, b])
    density = gaussian_kde(xy)(xy)
    z_order = np.argsort(density)
    ax.scatter(a[z_order], b[z_order], c=density[z_order], cmap='jet', rasterized=True)

In [180]:
labels = np.array(['Elliptical', 'Spiral', 'Starbust'])
plot_confusion_matrix(labels[t['TRUE_GALCLASS'][mask]], labels[t['GALAXY_CLASS'][mask]], normalize=False)

<AxesSubplot:title={'center':'Confusion Matrix'}, xlabel='Predicted label', ylabel='True label'>

<Figure size 720x432 with 2 Axes>

In [10]:
a, b = np.polyfit(t[mask]['HALPHA_LOGFLAM_EXT_MAG'], t['PHZ_PP_MODE_SFR'][mask], 1)

In [11]:
plt.figure()
density_plot(t[mask]['HALPHA_LOGFLAM_EXT_MAG'], t['PHZ_PP_MODE_SFR'][mask])
plt.xlim(-18.5, -15)
plt.ylim(-5, 3)
plt.xlabel('H$\\alpha$')
plt.ylabel('SFR')
plt.show()

<Figure size 720x432 with 1 Axes>

In [32]:
sb_bad_0 = t[(t['TRUE_GALCLASS'] == 2) & (t['GALAXY_CLASS'] == 0)]
sb_bad_1 = t[(t['TRUE_GALCLASS'] == 2) & (t['GALAXY_CLASS'] == 1)]
sb_good = t[(t['TRUE_GALCLASS'] == 2) & (t['GALAXY_CLASS'] == 2)]

In [33]:
def hist_mis(attr, ax=None):
    if ax is None:
        ax = plt.gca()
    bins = ax.hist(sb_bad_0[attr], histtype='step', label='Misclassified as elliptical', density=True)
    ax.hist(sb_bad_1[attr], histtype='step', label='Misclassified as spiral', bins=bins[1], density=True)
    ax.hist(sb_good[attr], histtype='step', label='Good', bins=bins[1], density=True)
    ax.set_xlabel(attr)
    ax.legend()

In [34]:
fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(20, 10))
for ax, attr in zip(axes.ravel(), ['SED_TEMPLATE', 'EBV', 'Z_OBS', 'REF_MAG', 'AV', 'EXT_LAW']):
    hist_mis(attr, ax)
plt.tight_layout()

<Figure size 1440x720 with 6 Axes>

In [35]:
fig, axes = plt.subplots(ncols=3, sharex=True, sharey=True, figsize=(20, 6))
bins = axes[0].hist2d(sb_bad_0['Z_OBS'], sb_bad_0['AV'], bins=20)
axes[0].set_title('Misclassified as elliptical')
axes[1].hist2d(sb_bad_1['Z_OBS'], sb_bad_1['AV'], bins=bins[1:3])
axes[1].set_title('Misclassified as spiral')
axes[2].hist2d(sb_good['Z_OBS'], sb_good['AV'], bins=bins[1:3])
axes[2].set_title('Good')

axes[0].set_ylabel('AV')
for ax in axes:
    ax.set_xlabel('Z_OBS')
plt.show()

<Figure size 1440x432 with 3 Axes>

In [89]:
fig, axes = plt.subplots(ncols=3, figsize=(20, 7))

    
axes[0].hist(t['SED_TEMPLATE'][(t['GALAXY_CLASS'] == 0) & mask], density=True)
axes[0].set_title('Classified as Elliptical')
axes[1].hist(t['SED_TEMPLATE'][(t['GALAXY_CLASS'] == 1) & mask], density=True)
axes[1].set_title('Classified as Spiral')
axes[2].hist(t['SED_TEMPLATE'][(t['GALAXY_CLASS'] == 2) & mask], density=True)
axes[2].set_title('Classified as Starbust')

for ax in axes:
    rell = patches.Rectangle((0, 0), SPIRAL_LMIN, 1., facecolor='red', alpha=0.2, zorder=-2, label='Elliptical')
    ax.add_artist(rell)
    rspi = patches.Rectangle((SPIRAL_LMIN, 0), SB_LMIN - SPIRAL_LMIN, 1., facecolor='yellow', alpha=0.2, zorder=-2, label='Spiral')
    ax.add_artist(rspi)
    rsb = patches.Rectangle((SB_LMIN, 0), 30 - SB_LMIN, 1., facecolor='blue', alpha=0.2, zorder=-2, label='Starbust')
    ax.add_artist(rsb)
    ax.set_xlabel('SED Template')
    
    ax.legend(handles=[rell, rspi, rsb])


plt.show()

<Figure size 1440x504 with 3 Axes>

# Reference sample

In [74]:
pp_data = np.load('/home/aalvarez/Work/Data/SC8/SC8R6.5/PP/pp_data_1.npy', mmap_mode='r')

In [83]:
plt.figure()
plt.hist(pp_data['GAL_TYPE'].mean(axis=1), bins=[0,1,2,3], align='left', density=True)
plt.show()

<Figure size 720x432 with 1 Axes>

In [85]:
plt.figure()
plt.hist(t['TRUE_GALCLASS'], bins=[0,1,2,3], align='left', density=True)
plt.show()

<Figure size 720x432 with 1 Axes>

In [90]:
plt.figure()
plt.hist(t['SED_TEMPLATE'], density=True)
plt.show()

<Figure size 720x432 with 1 Axes>

In [182]:
plt.figure()

cmap = colors.ListedColormap(cm.get_cmap("Dark2").colors[:3])

plt.scatter(
    t['SED_TEMPLATE'][mask], t['Z_OBS'][mask], c=t['GALAXY_CLASS'],
    marker='.', alpha=0.5, cmap=cmap,
    rasterized=True
)
plt.xlabel('SED Template')
plt.ylabel('True Z')

rell = patches.Rectangle((0, 0), SPIRAL_LMIN, 2.5, facecolor='red', alpha=0.2, zorder=-2, label='True Elliptical')
plt.gca().add_artist(rell)
rspi = patches.Rectangle((SPIRAL_LMIN, 0), SB_LMIN - SPIRAL_LMIN, 2.5, facecolor='yellow', alpha=0.2, zorder=-2, label='True Spiral')
plt.gca().add_artist(rspi)
rsb = patches.Rectangle((SB_LMIN, 0), 30 - SB_LMIN, 2.5, facecolor='blue', alpha=0.2, zorder=-2, label='True Starbust')
plt.gca().add_artist(rsb)

plt.legend(handles=[rell, rspi, rsb])
plt.colorbar(ticks=[0.33, 1., 1.66], label='Classification').ax.set_yticklabels(labels)

plt.show()

<Figure size 720x432 with 2 Axes>

In [174]:
plt.figure()

cmap = colors.ListedColormap(cm.get_cmap("Dark2").colors[:3])

plt.scatter(
    t['SED_TEMPLATE'][mask], t['EBV'][mask], c=t['GALAXY_CLASS'],
    marker='.', alpha=0.5, cmap=cmap,
    rasterized=True
)
plt.xlabel('SED Template')
plt.ylabel('EBV')

rell = patches.Rectangle((0, 0), SPIRAL_LMIN, 1., facecolor='red', alpha=0.2, zorder=-2, label='True Elliptical')
plt.gca().add_artist(rell)
rspi = patches.Rectangle((SPIRAL_LMIN, 0), SB_LMIN - SPIRAL_LMIN, 1., facecolor='yellow', alpha=0.2, zorder=-2, label='True Spiral')
plt.gca().add_artist(rspi)
rsb = patches.Rectangle((SB_LMIN, 0), 30 - SB_LMIN, 1., facecolor='blue', alpha=0.2, zorder=-2, label='True Starbust')
plt.gca().add_artist(rsb)

plt.legend(handles=[rell, rspi, rsb])
plt.colorbar(ticks=[0.33, 1., 1.66], label='Classification').ax.set_yticklabels(labels)

plt.show()

<Figure size 720x432 with 2 Axes>